## Set Up OpenAI gpt-4o-mini LLM

In [2]:
import os
import getpass

if "OPENAI_API_KEY" not in os.environ:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OPENAI_API_KEY: ")

Enter your OPENAI_API_KEY: ··········


In [3]:
import os

os.environ["OPENAI_API_BASE"] = "https://api.openai.com/v1"
os.environ["OPENAI_MODEL_NAME"] = "gpt-4o-mini"

In [4]:
from crewai import LLM

llm = LLM(
    model="openai/gpt-4o-mini",
    temperature=0
)

In [5]:
llm.call("Hi, How are you today?")

"Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?"

## Loading Tasks and Agents YAML files

In [6]:
import yaml

files = {
    'agents': '/content/agents.yaml',
    'tasks': '/content/tasks.yaml'
}

configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output

In [7]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Setting up Agents from CrewAI

In [12]:
from crewai import Agent

project_planning_agent = Agent(
  config=agents_config['project_planning_agent'],
  llm=llm
)

estimation_agent = Agent(
  config=agents_config['estimation_agent'],
  llm=llm
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent'],
  llm=llm
)

## Setting up Tasks using CrewAI

In [10]:
from crewai import Task

task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan
)

## Setting up Crew

In [11]:
from crewai import Crew

In [13]:
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

In [14]:
project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

In [15]:
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

In [16]:
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru

In [18]:
import pandas as pd

tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Create a responsive design,56.000000,['Bob Smith']
1,Implement a modern user interface,80.000000,['Bob Smith']
2,Develop a user-friendly navigation system,40.000000,['Jane Doe']
3,Include an 'About Us' page,24.000000,['Jane Doe']
4,Design a 'Services' page,32.000000,['Bob Smith']
5,Create a 'Contact Us' page,24.000000,['Jane Doe']
6,Implement a blog section,40.000000,['Jane Doe']
7,Ensure fast loading times and optimize for SEO,32.000000,['Jane Doe']
8,Integrate social media links,16.000000,['Jane Doe']
9,Include a testimonials section,24.000000,['Bob Smith']
